# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [190]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [191]:
filename = "../WeatherPy/weather_data.csv"
weather_data = pd.read_csv(filename)

weather_data.head()


,city,country,latitude,longitude,temp_max_F,humid,cloud,wind_mph,date_accessed
0,falmouth,GB,50.1544,-5.0711,83.30,85,0,3.00,21-07-2021
1,bezhanitsy,RU,56.9767,29.8907,67.15,45,94,8.72,21-07-2021
2,hami,CN,42.8000,93.4500,91.80,19,95,2.75,21-07-2021
3,chokurdakh,RU,70.6333,147.9167,44.78,78,42,6.69,21-07-2021
4,alyangula,AU,-13.8483,136.4192,68.43,71,16,9.06,21-07-2021


In [192]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           567 non-null    object 
 1   country        562 non-null    object 
 2   latitude       567 non-null    float64
 3   longitude      567 non-null    float64
 4   temp_max_F     567 non-null    float64
 5   humid          567 non-null    int64  
 6   cloud          567 non-null    int64  
 7   wind_mph       567 non-null    float64
 8   date_accessed  567 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 40.0+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [193]:
# Store latitude and longitude in locations
locations = weather_data[["latitude", "longitude"]]

# use humidity as the weight
map_weight = weather_data["humid"]

# set figure layout
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px'
}

# Plot Heatmap
fig = gmaps.figure(map_type = 'HYBRID', layout=figure_layout)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                weights=map_weight,
                                dissipating=False, 
                                max_intensity=200,
                                point_radius=8, 
                                opacity = 0.4,
                                gradient = [
                                            'white',
                                            'aqua',
                                            'purple'
                                            ])

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', padding='1px', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [215]:
ideal_weather_places = weather_data.loc[
    (weather_data['temp_max_F']> 81) &
    (weather_data['temp_max_F']< 85) &
    (weather_data['cloud']<20) & 
    (weather_data['humid']<90) &
    (weather_data['humid']>60) &
    (weather_data['wind_mph']<10)]


ideal_weather_places.reset_index(inplace = True, drop = True)
ideal_weather_places

,city,country,latitude,longitude,temp_max_F,humid,cloud,wind_mph,date_accessed
0,falmouth,GB,50.1544,-5.0711,83.30,85,0,3.00,21-07-2021
1,boueni,YT,-12.9025,45.0761,82.38,70,18,9.73,21-07-2021
2,nhamunda,BR,-2.1861,-56.7131,83.35,69,0,3.09,21-07-2021
3,nanchong,CN,30.7951,106.0847,82.47,76,2,2.57,21-07-2021
4,katsuura,JP,35.1333,140.3000,82.11,88,9,5.91,21-07-2021
5,tutoia,BR,-2.7619,-42.2744,83.66,62,7,9.66,21-07-2021
6,amapa,BR,1.0000,-52.0000,81.09,74,15,3.96,21-07-2021
7,tateyama,JP,34.9833,139.8667,82.02,85,14,5.39,21-07-2021
8,margate,GB,51.3813,1.3862,84.18,76,4,9.28,21-07-2021
9,cabanas,CU,22.9758,-82.9219,84.67,78,10,3.91,21-07-2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [216]:
hotel_df = ideal_weather_places[['city', 'latitude', 'longitude']]
hotel_df = hotel_df.rename(columns = {"city":"City", "latitude":"Lat", "longitude":"Lng"})

# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""

In [217]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "keyword" : "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add location to params dict, using latitude and longitude columns
    params['location'] = (f'{str(row["Lat"])},{str(row["Lng"])}')

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

        
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")



Retrieving Results for Index 0: falmouth.
Closest hotel is Seaview Inn.
------------
Retrieving Results for Index 1: boueni.
Closest hotel is The feet in water.
------------
Retrieving Results for Index 2: nhamunda.
Closest hotel is Porto de Nhamundá.
------------
Retrieving Results for Index 3: nanchong.
Closest hotel is Beihu Hotel （East Gate）.
------------
Retrieving Results for Index 4: katsuura.
Closest hotel is Minshuku Magusa.
------------
Retrieving Results for Index 5: tutoia.
Closest hotel is Salinas Pousada.
------------
Retrieving Results for Index 6: amapa.
Closest hotel is Izak.
------------
Retrieving Results for Index 7: tateyama.
Closest hotel is tu.ne.Hostel（ツネホステル）.
------------
Retrieving Results for Index 8: margate.
Closest hotel is Sands Hotel Margate.
------------
Retrieving Results for Index 9: cabanas.
Closest hotel is Terrazas Atlántico.
------------


In [218]:
# find the Country that the city is in
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params = {"key": g_key, "result_type":"country"}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add location to params dict, using latitude and longitude columns
    params['latlng'] = (f'{str(row["Lat"])},{str(row["Lng"])}')

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

        
    try:
               
        hotel_df.loc[index, 'Country'] = results[0]['address_components'][0]['long_name']
            
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: falmouth.
------------
Retrieving Results for Index 1: boueni.
------------
Retrieving Results for Index 2: nhamunda.
------------
Retrieving Results for Index 3: nanchong.
------------
Retrieving Results for Index 4: katsuura.
------------
Retrieving Results for Index 5: tutoia.
------------
Retrieving Results for Index 6: amapa.
------------
Retrieving Results for Index 7: tateyama.
------------
Retrieving Results for Index 8: margate.
------------
Retrieving Results for Index 9: cabanas.
------------


In [219]:
hotel_df

,City,Lat,Lng,Hotel Name,Country
0,falmouth,50.1544,-5.0711,Seaview Inn,United Kingdom
1,boueni,-12.9025,45.0761,The feet in water,Mayotte
2,nhamunda,-2.1861,-56.7131,Porto de Nhamundá,Brazil
3,nanchong,30.7951,106.0847,Beihu Hotel （East Gate）,China
4,katsuura,35.1333,140.3000,Minshuku Magusa,Japan
5,tutoia,-2.7619,-42.2744,Salinas Pousada,Brazil
6,amapa,1.0000,-52.0000,Izak,Brazil
7,tateyama,34.9833,139.8667,tu.ne.Hostel（ツネホステル）,Japan
8,margate,51.3813,1.3862,Sands Hotel Margate,United Kingdom
9,cabanas,22.9758,-82.9219,Terrazas Atlántico,Cuba


In [220]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [221]:
# Reproduce heat map from above
locations = weather_data[["latitude", "longitude"]]
map_weight = weather_data["humid"]
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, 
                                weights=map_weight,
                                dissipating=False, 
                                max_intensity=200,
                                point_radius=6, 
                                opacity = 0.4,
                                gradient = [
                                            'white',
                                            'aqua',
                                            'purple'
                                            ])

fig.add_layer(heat_layer)

# Add marker layer ontop of heat map

# Create a marker_layer using the hotel_info to fill the info box
markers = gmaps.marker_layer(locations = locations2, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))